## Dataset

In [1]:
!pip install scipy

In [2]:
from imb_datasets.data_loader import make_longtailed_imb, get_imbalanced
from torchvision import datasets, transforms

In [3]:
N_SAMPLES_PER_CLASS_BASE = make_longtailed_imb(max_num=5000, class_num=10, gamma=50)

0.6474788028695254


In [4]:
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
])
transform_test = transforms.Compose([
    transforms.ToTensor(),
])

In [7]:
train_loader, val_loader, test_loader = get_imbalanced(
    dataset="cifar10",
    num_sample_per_class=N_SAMPLES_PER_CLASS_BASE,
    batch_size=128,
    TF_train=transform_train,
    TF_test=transform_test,
    download=True,
)

Building CV cifar10 data loader with 8 workers
Files already downloaded and verified
Files already downloaded and verified


## Model

In [12]:
from networks import WideResNet

In [13]:
net = WideResNet(depth=28, num_classes=10, widen_factor=10)

In [14]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

In [15]:
count_parameters(net)

36479194

In [26]:
net = net.cuda()

## Wandb

In [34]:
import wandb
wandb.login()

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [35]:
wandb.init(
    project="pure-noise",
)

wandb: Currently logged in as: seungjaeryanlee. Use `wandb login --relogin` to force relogin


In [ ]:
wandb.config.update({
    # Training
    "n_epoch": 200,
    # Optimizer
    "optim__lr": 0.1,
    "optim__momentum": 0.9,
    "optim__weight_decay": 2e-4,
    # Model
    "model__wideresnet__depth": 28,
    "model__wideresnet__k": 10,
})

## Training Loop

In [27]:
import torch.nn as nn
loss_fn = torch.nn.CrossEntropyLoss()

In [28]:
import torch.optim as optim
optimizer = optim.SGD(net.parameters(), lr=0.1, momentum=0.9, weight_decay=2e-4)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.01)

In [31]:
%%timeit
N_EPOCH = 200

for epoch_i in range(N_EPOCH):
    net.train()

    # Training Phase
    train_loss = 0
    for minibatch_i, (inputs, labels) in enumerate(train_loader):
        inputs = inputs.cuda()
        labels = labels.cuda()
        
        outputs = net(inputs)
        loss = loss_fn(outputs, labels)
        loss.backward()
        
        optimizer.step()
        train_loss += loss

    # Validation Phase
    with torch.no_grad():
        val_loss = 0
        for minibatch_i, (inputs, labels) in enumerate(val_loader):
            inputs = inputs.cuda()
            labels = labels.cuda()

            outputs = net(inputs)
            loss = loss_fn(outputs, labels)
            val_loss += loss

    wandb.log({
        "train_loss": train_loss,
        "val_loss": val_loss,
    })
    if epoch_i in [160, 180]:
        scheduler.step()

Minibatch   0/110 | Loss 1.11787748336792
Minibatch   1/110 | Loss 1.0631237030029297
Minibatch   2/110 | Loss 0.9617557525634766
Minibatch   3/110 | Loss 1.3260769844055176
Minibatch   4/110 | Loss 1.147119402885437
Minibatch   5/110 | Loss 1.0742428302764893
Minibatch   6/110 | Loss 1.1264724731445312
Minibatch   7/110 | Loss 1.0463430881500244
Minibatch   8/110 | Loss 1.1403892040252686
Minibatch   9/110 | Loss 3.3802103996276855
Minibatch  10/110 | Loss 1.4045207500457764
Minibatch  11/110 | Loss 1.1303356885910034


Traceback (most recent call last):
  File "/opt/conda/lib/python3.7/multiprocessing/queues.py", line 242, in _feed
    send_bytes(obj)
  File "/opt/conda/lib/python3.7/multiprocessing/connection.py", line 200, in send_bytes
    self._send_bytes(m[offset:offset + size])
  File "/opt/conda/lib/python3.7/multiprocessing/connection.py", line 404, in _send_bytes
    self._send(header + buf)
  File "/opt/conda/lib/python3.7/multiprocessing/connection.py", line 368, in _send
    n = write(self._handle, buf)
BrokenPipeError: [Errno 32] Broken pipe
Traceback (most recent call last):
  File "/opt/conda/lib/python3.7/multiprocessing/queues.py", line 242, in _feed
    send_bytes(obj)
  File "/opt/conda/lib/python3.7/multiprocessing/connection.py", line 200, in send_bytes
    self._send_bytes(m[offset:offset + size])
  File "/opt/conda/lib/python3.7/multiprocessing/connection.py", line 404, in _send_bytes
    self._send(header + buf)
  File "/opt/conda/lib/python3.7/multiprocessing/connection.py", 

KeyboardInterrupt: 